In [ ]:
from my_functions import *

In [ ]:
set_db()

# Initialize some data

Let's create three users and save them:

In [ ]:
martin = {'name': 'Martin Marsal', 'tweets': [], 'followers': [], 'timeline': []}
christian = {'name': 'Christian Diegmann', 'tweets': [], 'followers': [], 'timeline': []}
robin = {'name': 'Robin Schüle', 'tweets': [], 'followers': [], 'timeline': []}

In [ ]:
save_user(martin)

In [ ]:
save_user(christian)

In [ ]:
save_user(robin)

Let's add the other users to each followers list.

In [ ]:
add_to_followers(martin['_id'], christian['_id'])

In [ ]:
add_to_followers(christian['_id'], robin['_id'])

In [ ]:
add_to_followers(robin['_id'], martin['_id'])

# 1st access pattern: Post a tweet

In [ ]:
tweet = {'_id': ObjectId(), 'text': 'Moin, moin.', 'likes': 0, 'replies': []}
tweet_2 = {'_id': ObjectId(), 'text': 'MongoDB ist super!', 'likes': 0, 'replies': []}

In [ ]:
newMartin = find_user(martin['_id'])

In [ ]:
post_tweet(newMartin, tweet)
post_tweet(newMartin, tweet_2)

In [ ]:
class TestPostTweet(unittest.TestCase):
    def test_post_tweet_enriched_timeline(self):
        timeline = read_timeline(christian)
        timeline_length = len(timeline['timeline'])
        self.assertEqual(timeline_length, 2)

    def test_post_tweet_empty_timeline(self):
        timeline = read_timeline(robin)
        timeline_length = len(timeline['timeline'])
        self.assertEqual(timeline_length, 0)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 2nd access pattern: Post a reply

In [ ]:
reply = {'_id': ObjectId(), 'text': 'Hallo zurück.', 'likes': 0}

In [ ]:
newMartin = find_user(martin['_id'])

In [ ]:
post_reply(newMartin['tweets'][0], reply)

In [ ]:
class TestPostReply(unittest.TestCase):
    def test_post_reply_enriched_replies(self):
        timeline = read_timeline(christian)
        replies_length = len(timeline['timeline'][0]['replies'])
        self.assertEqual(replies_length, 1)

    def test_post_reply_empty_replies(self):
        timeline = read_timeline(christian)
        replies_length = len(timeline['timeline'][1]['replies'])
        self.assertEqual(replies_length, 0)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostReply)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 3rd access pattern: Edit a tweet

In [ ]:
newMartin = find_user(martin['_id'])

In [ ]:
edit_tweet(newMartin['tweets'][0])

In [ ]:
class TestEditTweet(unittest.TestCase):
    user = find_user(newMartin['_id'])
    
    def test_edit_tweet_edited_tweet(self):
        text = self.user['tweets'][0]['text']
        self.assertEqual(text, 'Moin, moin. Wie geht es euch?')

    def test_edit_tweet_unedited_tweet(self):
        text = self.user['tweets'][1]['text']
        self.assertEqual(text, 'MongoDB ist super!')

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestEditTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 4th access pattern: Read a timeline

In [ ]:
newChristian = find_user(christian['_id'])

In [ ]:
read_timeline(newChristian)

In [ ]:
class TestReadTimeline(unittest.TestCase):
    def test_read_timeline(self):
        timeline = read_timeline(christian)
        timeline_length = len(timeline['timeline'])
        self.assertEqual(timeline_length, 2)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestReadTimeline)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 5th access pattern: Delete a Tweet

In [ ]:
newMartin = find_user(martin['_id'])

In [ ]:
delete_tweet(newMartin, tweet)

In [ ]:
class TestDeleteTweet(unittest.TestCase):
    user = find_user(newMartin['_id'])
    def test_deleted_tweet_in_follower_timeline(self):
        timeline = read_timeline(christian)
        timeline_length = len(timeline['timeline'])
        self.assertEqual(timeline_length, 1)
    def test_deleted_tweet_in_user_tweets(self):
        tweets_length = len(self.user['tweets'])
        self.assertEqual(tweets_length, 1)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestDeleteTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 6th acces pattern: Delete user

In [ ]:
newMartin = find_user(martin['_id'])

In [ ]:
delete_user(newMartin)

In [ ]:
class TestDeleteUser(unittest.TestCase):
    user = find_user(martin['_id'])
    
    def test_delete_user(self):
        self.assertEqual(self.user, False)

    def test_delete_tweet(self):
        timeline = read_timeline(christian)
        timeline_length = len(timeline['timeline'])
        self.assertEqual(timeline_length, 0)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestDeleteUser)
    unittest.TextTestRunner(verbosity=2).run(suite)